In [ ]:
import os
import numpy as np
import pandas as pd
from cmdstanpy import CmdStanModel

def set_formats(digits: int, width: int) -> None:
  """ set display precision, output width """
  np.set_printoptions(precision=digits)
  np.set_printoptions(suppress=True)
  np.set_printoptions(threshold=np.inf)
  pd.set_option('display.precision', digits)
  format_string = '{{:.{}f}}'.format(digits)
  pd.options.display.float_format = format_string.format
  pd.set_option('display.max_rows', None)
  pd.set_option('display.max_columns', None)
  pd.set_option('display.width', width)

def summary_by_var(summary_df: pd.DataFrame, varname: str) -> None:
    row_names = list(summary_df.index)
    var_rows = [row_names.index(name) for name in row_names if name.startswith(varname)]
    print(summary_df.iloc[var_rows,:])

set_formats(2, 100)

In [ ]:
# plotting libs
import matplotlib.pyplot as plt
import plotnine as p9

# suppress plotnine warnings
import warnings
warnings.filterwarnings('ignore')

# setup plotnine look and feel
p9.theme_set(
  p9.theme_grey() + 
  p9.theme(text=p9.element_text(size=10),
        plot_title=p9.element_text(size=14),
        axis_title_x=p9.element_text(size=14),
        axis_title_y=p9.element_text(size=14),
        axis_text_x=p9.element_text(size=12),
        axis_text_y=p9.element_text(size=12)
       )
)
xlabels_90 = p9.theme(axis_text_x = p9.element_text(angle=90, hjust=1))

Can we fit the Indiana data to simple Stan models; run posterior predictive check

In [ ]:
df = pd.read_csv("data/data_tests_sex_age_eth_time.csv")
in_data = df.to_dict(orient='list')

in_data['N_age'] = max(in_data['age'])
in_data['N_eth'] = max(in_data['eth'])
in_data['N_time'] = max(in_data['time'])
in_data['N'] = len(in_data['tests'])

in_data['sens'] = 0.7
in_data['spec']=0.995
in_data['intercept_prior_mean']=-4
in_data['intercept_prior_scale']=2.5


In [ ]:
for var, value in in_data.items():
    print(var, value)

In [ ]:
naive_4_mod = CmdStanModel(stan_file="stan/binomial_4preds_naive.stan")

naive_4_path = naive_4_mod.pathfinder(data=in_data)
path_inits = naive_4_path.create_inits()
naive_4_fit = naive_4_mod.sample(data=in_data, inits=path_inits, iter_warmup=1500)

In [ ]:
print(naive_4_fit.diagnose())

In [ ]:
n4_summary = naive_4_fit.summary()
print(summary_by_var(n4_summary, "alpha"))
print(summary_by_var(n4_summary, "sigma"))
print(summary_by_var(n4_summary, "beta_age["))
print(summary_by_var(n4_summary, "beta_eth["))
print(summary_by_var(n4_summary, "beta_time["))

In [ ]:
s2z_4_mod = CmdStanModel(stan_file="stan/binomial_4preds_s2zh.stan")

In [ ]:
if (in_data['sex'][0] == 0):
    in_data['sex'] = [x + 1 for x in in_data['sex']]

In [ ]:
s2z_4_path = s2z_4_mod.pathfinder(data=in_data)
path_inits = s2z_4_path.create_inits()
s2z_4_fit = s2z_4_mod.sample(data=in_data, inits=path_inits)

In [ ]:
s2z4_summary = s2z_4_fit.summary()
print(summary_by_var(s2z4_summary, "alpha"))
print(summary_by_var(s2z4_summary, "sigma"))
print(summary_by_var(s2z4_summary, "beta_sex["))
print(summary_by_var(s2z4_summary, "beta_age["))
print(summary_by_var(s2z4_summary, "beta_eth["))
print(summary_by_var(s2z4_summary, "beta_time["))

In [ ]:
y_rep_pd = naive_4_fit.draws_pd(vars='y_rep').iloc[:, -30:]
s2z_4_y_rep_pd = s2z_4_fit.draws_pd(vars='y_rep').iloc[:, -30:]

true_y = {col: in_data['pos_tests'][i] for i, col in enumerate(y_rep_pd.columns)}
y_rep_long = y_rep_pd.melt(var_name='y_rep', value_name='value')
y_rep_long['y'] = y_rep_long['y_rep'].map(true_y)

s2z_4_y_rep_long = s2z_4_y_rep_pd.melt(var_name='y_rep', value_name='value')
s2z_4_y_rep_long['y'] = s2z_4_y_rep_long['y_rep'].map(true_y)

(p9.ggplot()
    + p9.geom_boxplot(data=y_rep_long, mapping=p9.aes(x='y_rep',y='value'), width=1, alpha=0.5, outlier_alpha=0.2, color='darkblue')
    + p9.geom_boxplot(data=s2z_4_y_rep_long, mapping=p9.aes(x='y_rep',y='value'), width=0.85, alpha=0.5, outlier_alpha=0.2, color='darkorange')
    + p9.geom_point(data=y_rep_long, mapping=p9.aes(x='y_rep', y='y'), color='purple', size=5)
    + xlabels_90
    + p9.theme(figure_size=(24,16))
)

Hard sum-to-zero constraint provides higher ESS, no divergences, all R-hat values 1.00.